# Tutorial 2: object detection

This example demonstrates basic `phenopype` functionality by showing how multiple objects can be found and extracted from images via thresholding. The images used for this tutorial contain isopods sitting inside a white tray underneath a camera stand, photopgraphed with a 50 mm lens on a Canon 750D. 
***
* [collect images](#collect)
* [set a scale](#scale)
* [object finder - single object](#object1)
* [mask images](#mask)
* [object finder - multiple objects](#object2)
* [object finder - multiple files](#object3)
* [object finder - variable background](#object4)



In [1]:
import os
import phenopype as pp

# for this tutorial, you should be in the "tutorial directory of phenopype-master"
#os.getcwd()
#os.chdir("tutorials")

# collect images<a name="collect"></a>

Start by loading the program. We can use `import as` with shorter bindings, so we can type `pp` instead of `phenopype` everytime we call a function. Next we specificy the folder containing the images, making a project object that contains all the image names and paths. The function `project_maker` will then collect all files that match your specifications (filetypes or names). 

For this first simple example, we only want to include the image named "bug1.jpg". We can do so using `include` (or, `exclude` to skip images whose filenames contain this string). Note that this can be any sub-string and does not have to be the whole filename, so in this case "bug1" will suffice.

Let's have a quick look at the image using the collected absolute filepaths (accessible from the `my_proj` object) and the _phenopye_ function `show_img` (contains all the `opencv` GUI controls explained in  [tutorial 1](1_python_intro.ipynb#images). Close the image by hitting enter or closing the window:

In [2]:
my_proj = pp.project_maker(image_dir = "images", include=["bug1"]) 
 
# HINT: image_dir can be relative or absolute 
# e.g. something like "your_download_directory//phenopype-master//tutorials//images"



----------------------------------------------------------------
Project settings - "My project, 2019-02-16":


Image directory: images
Mode: dir
Filetypes: []
Include:['bug1']
Exclude: []
----------------------------------------------------------------
Filenames: 
['bug1_with_scale.jpg']


In [3]:
img_path = my_proj.filepaths[0] 
pp.show_img(img_path)

## setting a scale <a name="scale"></a>
-|-
-|-
![](../assets/tutorials/make_scale.gif)|![](../assets/tutorials/scale.png)

Our image contains and organism, and a scale. To make our measurements meaningful, we need to know the the pixel to mm ratio. To do so we we load the image into the `scale_maker`, `zoom` into the scale area for better visibility, and mark the distance we specifiy - in this case 10 mm. 



In [4]:
scale = pp.scale_maker(image=img_path, value=10, unit="mm",  zoom=True, show=True)


Mark the outline of the scale by left clicking, remove points by right clicking, finish with enter.
Finished, scale outline drawn. Now dd the scale by clicking on two points with a known distance between them:
Adding point 1 of 2 to scale
Adding point 2 of 2 to scale


------------------------------------------------
Finished - your scale has 549 pixel per 10 mm.
------------------------------------------------




We can access the scale by calling its variable `measured`, which always will give you the ratio of **pixels per 1 mm**.

In [5]:
print(scale.measured, " # pixel per mm")

54.9  # pixel per mm


## object finder <a name="object-finder"></a>

Finding objects inside an image is a core function of _phenopype_. Results depend on how "good" your pictures are with respect to foreground contrast of your objects, homogeneity of your background, and overall illumination and resoution. The pictures we have loaded above are pretty good for that purpose. However, don't give up just now if your images have less ideal contrast or a noisy background. _phenopype_ comes with a flavor of different preprossing strategies and well established thresholding algorithms in [opencv](https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html), wrapped in a simple method: the `object_finder`.

In [6]:
help(pp.object_finder)

Help on class object_finder in module phenopype.base:

class object_finder(builtins.object)
 |  object_finder(image)
 |  
 |  Initialize object finder class, loads image.
 |  
 |  Parameters
 |  ----------
 |  
 |  image: str or array
 |      absolute or relative path to OR numpy array of image containing the objects
 |  
 |  Methods defined here:
 |  
 |  __init__(self, image)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  find_objects(self, **kwargs)
 |      Method in object finder class: find objects in colour or grayscale images using thresholding
 |      
 |      Parameters
 |      ----------
 |      thresholding: list (default: ["otsu"])
 |          determines the type of thresholding: 
 |              - "binary" needs an interger for the threshold value (default: 127), 
 |              - "adaptive" needs odd integer for blocksize (default: 99) and constant to be subtracted (default 1) 
 |              - for more info see https://docs.opencv.org/3

`object_finder` is a [class](https://docs.python.org/3.7/tutorial/classes.html), that means we have to initialize it first, and then run the actual [method](https://stackoverflow.com/questions/20981789/difference-between-methods-and-functions-in-python-compared-to-c). After initializing it with our image, we can find objects using `find_objects`.

In [7]:
of = pp.object_finder(image=img_path)
results = of.find_objects()

Warning - no scale specified
     diameter  area
idx                
1           7    24
2          19    67
3          15     8
4          17    16
5          13    16
..        ...   ...
169        48   371
170        24    46
171        74   111
172        12    19
173         9    13

[173 rows x 2 columns]


----------------------------------------------------------------
Found 1259 objects in bug1_with_scale.jpg:
  ==> 173 are valid objects
    - 1086 are noise
----------------------------------------------------------------


### masking images <a name="mask"></a>

Ok, this is not right - we need to only include our organism in our image by applying a mask to be included. This we can do with the function called `polygon_maker` and the argument `include=True`

In [11]:
mask = pp.polygon_maker(img_path)
mask1 = mask.draw(mode="rectangle", show=True, include=True)


Mark the outline of your arena, i.e. what you want to include in the image analysis by left clicking, finish with enter.


The mask can the be passed on the the `object_finder`:

In [12]:
results = of.find_objects(mask=[mask1]) # NOTE: your mask should be inside a list -> []

Warning - no scale specified
     diameter   area
idx                 
1           9      5
2          28     97
3          11     27
4          11     11
5          34     89
..        ...    ...
13        746  78746
14          6     12
15         43    588
16         40    467
17         80   1020

[17 rows x 2 columns]


----------------------------------------------------------------
Found 77 objects in bug1_with_scale.jpg:
  ==> 17 are valid objects
    - 60 are noise
----------------------------------------------------------------


The next problem is, that our organism is being recognized as many small objects. If we only have one organism in our image, we can swith to the `mode` "single". Also, we want to disregard legs and antenna. The removal of small structures can be accomplished by adding some gaussian noise to the image with the `blur1` variable (`blur1` = first pass blurring, `blur2` = second pass blurring). The provided number of the size of your blur kernel in pixels (bigger = more blurred). At this point we should also include the scale we measured earlier:

In [13]:
results = of.find_objects(mask=[mask1], mode="single", blur1=25, scale=scale.measured) 

----------------------------------------------------------------
Found following object in bug1_with_scale.jpg:
----------------------------------------------------------------
     diameter   area
idx                 
1         735  80373


What we see in the console is just for evalution and does not show us much information. To see at what we actually measured, let's look at the `results` object, which contains some metadata, and the phenotypic information.

In [14]:
results

,date_taken,date_analyzed,idx,resize_factor,x,y,scale,diameter,area
filename,,,,,,,,,
bug1_with_scale.jpg,2017-12-12 16:15:32,2019-02-16 00:05:59,1,1,2590,2444,54.9,735,80373




Variable|Description
-|-
filename | name of the image file
date_taken | timestamp of when your image was taken. (y-m-d h-m-s), or NA
date_analyzed | timestamp of when your image was analyzed (i.e., current time). (y-m-d h-m-s)
idx | if you have multiple objects in your image, this will correspond to the labels
resize_factor | sometimes it is necessary to resize large images. this keeps track of it 
scale | the provided scale, number of pixels per 1 mm
diameter | from the bounding circle of our object
area | inside the contour of our object


We can measure aditional parameters that we pass on using the `operations` argument - e.g. "bgr", which returns the mean values for blue, green, and red pixels (for a full list of operations, see `help(pp.object_finder)`).

In [15]:
results = of.find_objects(mask=[mask1], mode="single", blur1=25, scale=scale.measured, operations = ["grayscale", "bgr"])
results

----------------------------------------------------------------
Found following object in bug1_with_scale.jpg:
----------------------------------------------------------------
     diameter   area
idx                 
1         735  80373


,date_taken,date_analyzed,idx,resize_factor,x,y,scale,diameter,area,grayscale_mean,grayscale_sd,bgr_mean,bgr_sd
filename,,,,,,,,,,,,,
bug1_with_scale.jpg,2017-12-12 16:15:32,2019-02-16 00:06:04,1,1,2590,2444,54.9,735,80373,74,37,"(40, 64, 107)","(36, 39, 35)"


We can keep running the `find_objects` method until we are happy with our results. Once that is the case, we save both the results data frame using `save_csv`; and the processed image, which we get by accessing `image_processed` in the `object_finder` object, and saving it with `save_img`. As name we use the filename that we get from the `my_proj` object.

Note that by default, text files and images are overwritten if they already exists in the specified directory. Also, if a directory does not exists, it will be created.

In [16]:
img_name = my_proj.filenames[0]

pp.save_csv(df=results, name=img_name, save_dir="images_out")
pp.save_img(image=of.image_processed, name=img_name, save_dir="images_out", resize=0.25)
# NOTE: you can resize images with the "resize" argument to save space

You now should be able to handle the `object_finder` class. Below I will introduce more examples and processing steps that can improve results.

## object finder - multiple objects <a name="object2"></a>

For this example we will load a different set of images.

In [17]:
my_proj = pp.project_maker(image_dir = "images", include=["multiple"]) 



----------------------------------------------------------------
Project settings - "My project, 2019-02-16":


Image directory: images
Mode: dir
Filetypes: []
Include:['multiple']
Exclude: []
----------------------------------------------------------------
Filenames: 
['isopods_multiple_1.jpg', 'isopods_multiple_2.jpg', 'isopods_multiple_3.jpg']


In [18]:
img_path = my_proj.filepaths[0] # we only use the first image
scale = pp.scale_maker(image=img_path, value=10, unit="mm",  zoom=True, show=True) # measure scale


Mark the outline of the scale by left clicking, remove points by right clicking, finish with enter.
Finished, scale outline drawn. Now dd the scale by clicking on two points with a known distance between them:
Adding point 1 of 2 to scale
Adding point 2 of 2 to scale


------------------------------------------------
Finished - your scale has 137 pixel per 10 mm.
------------------------------------------------




In [19]:
of = pp.object_finder(img_path)
results = of.find_objects(scale=scale.measured) 

     diameter    area
idx                  
1          16      53
2          14      18
3          34     187
4           9      21
5          42     171
..        ...     ...
120        20      60
121        13      29
122         7       2
123      3604  788847
124       262    4084

[124 rows x 2 columns]


----------------------------------------------------------------
Found 742 objects in isopods_multiple_1.jpg:
  ==> 124 are valid objects
    - 618 are noise
----------------------------------------------------------------


Looks terrible. We need to exclude areas, and pass the mask on to `find_objects`.

In [21]:
mask = pp.polygon_maker(img_path)
mask1 = mask.draw(label="tray", mode="rectangle", include=True) # include the tray area
mask2 = mask.draw(label="scale", mode="rectangle", show=True, include=False) # exclude the scale inside the tray area


Mark the outline of your arena, i.e. what you want to include in the image analysis by left clicking, finish with enter.

Mark the outline of your arena, i.e. what you want to include in the image analysis by left clicking, finish with enter.


In [22]:
results = of.find_objects(mask=[mask1, mask2], scale=scale.measured)

     diameter  area
idx                
1          16    53
2          14    18
3          34   187
4           9    21
5          42   171
..        ...   ...
78         13    46
79        110  1109
80         86   481
81         17    98
82         93   568

[82 rows x 2 columns]


----------------------------------------------------------------
Found 619 objects in isopods_multiple_1.jpg:
  ==> 82 are valid objects
    - 537 are noise
----------------------------------------------------------------


Still not great, there is a lot of noise and some bugs are not identified properly. We can try some blurring and implementing a minimum diameter.

In [23]:
results = of.find_objects(mask=[mask1, mask2], scale=scale.measured, min_diam=10, blur1=10)

     diameter  area
idx                
1          30   176
2          32   203
3         100  1070
4         106  2871
5          91  1591
..        ...   ...
22        109  1364
23         57   418
24         12    58
25         11    49
26         86   780

[26 rows x 2 columns]


----------------------------------------------------------------
Found 34 objects in isopods_multiple_1.jpg:
  ==> 26 are valid objects
    - 6 are noise
    - 2 are not bigger than minimum diameter
----------------------------------------------------------------


Blurring tends to "eat away" the object borders. We can counteract this by adding a "correction factor" that will add some more area to our objects. `corr_factor` is a list that takes three arguments: [shape, value, iterations]. See `help(object_finder)` for details. 

Since we add more "flesh to the bone", we can also increase the minimum diameter of our objects and also introduce `min_area` to get rid of those small particles. Note that the preview that gets put out after every `find_object` run can help you specify those parameters. E.g. if most objects have around 100 pixels and an area >1000 pixels, we want to set the threshold so that the majority stays in, but the smallest objects get excluded.

In [24]:
results = of.find_objects(mask=[mask1, mask2], scale=scale.measured, min_diam=20,min_area=1100, blur1=10, corr_factor=["ellipse",10,1])


     diameter  area
idx                
1          96  1429
2         109  2141
3         117  4116
4         101  2650
5         102  2217
..        ...   ...
16         98  1778
17         96  2095
18        119  2648
19        111  1722
20         95  1706

[20 rows x 2 columns]


----------------------------------------------------------------
Found 26 objects in isopods_multiple_1.jpg:
  ==> 20 are valid objects
    - 4 are not bigger than minimum diameter
    - 6 are not bigger than minimum area
----------------------------------------------------------------


Ok, this looks good, let's save the results and the processed image for reference, the `idx` column corresponds to the labels inside the image. 

In [25]:
img_name = my_proj.filenames[0]

pp.save_csv(df=results, name=img_name, save_dir="images_out")
pp.save_img(image=of.image_processed, name=img_name, save_dir="images_out", resize=0.5)

## object finder - multiple files <a name="object3"></a>

Let's do this for all the files in our project. Because the tray on which the bugs sit isn't perfectly centered in each picture, we should mark the boundaries with the `polygon_maker`. However, because the scale is the same in each picture, we can use the `detect` method from the `scale_maker` class. `detect` uses the accelerated KAZE (AKAZE: 
https://www.youtube.com/watch?v=lI50PGr2TEU) algorithm for feature registration and matching. I will expand more on this in a future version of the tutorial.

In [26]:
# MAKE FILE LIST

my_proj = pp.project_maker(image_dir = "images", include=["multiple"]) 



----------------------------------------------------------------
Project settings - "My project, 2019-02-16":


Image directory: images
Mode: dir
Filetypes: []
Include:['multiple']
Exclude: []
----------------------------------------------------------------
Filenames: 
['isopods_multiple_1.jpg', 'isopods_multiple_2.jpg', 'isopods_multiple_3.jpg']


In [27]:
# MARK SCALE ONCE, GET PIXEL RATIO

img_path = my_proj.filepaths[0] # we only use the first image
scale = pp.scale_maker(image=img_path, value=10, unit="mm",  zoom=True, show=True) # measure scale


Mark the outline of the scale by left clicking, remove points by right clicking, finish with enter.
Finished, scale outline drawn. Now dd the scale by clicking on two points with a known distance between them:
Adding point 1 of 2 to scale
Adding point 2 of 2 to scale


------------------------------------------------
Finished - your scale has 137 pixel per 10 mm.
------------------------------------------------




In [28]:
# MARK TRAY AREA ONLY, SCALE WILL BE AUTOMATICALLY DETECTED. CONTINUE WITH ENTER

# loop through both the projects filepaths and filenames using "zip":
for path, name in zip(my_proj.filepaths, my_proj.filenames):
           
    # mark the tray for each image. if your picture doesn't change dramatically, you can skip this
    arena = pp.polygon_maker(path)
    mask1 = arena.draw(label="tray", mode="rectangle", show=True)
    
    # this is a scale detector! if your scale is identical, the scale-finder will find it!
    mask2, scale.current = scale.detect(path, min_matches=10, show=True)    

    # we use the optimal settings we found above
    of = pp.object_finder(path)
    results = of.find_objects(mask=[mask1, mask2], scale=scale.measured, min_diam=20,min_area=1100, blur1=10, corr_factor=["ellipse",10,1])
        
    # save after every iteration
    pp.save_csv(results, name, "tutorials\\images_out")
    pp.save_img(of.image_processed, name, "tutorials\\images_out", resize=0.5)


Mark the outline of your arena, i.e. what you want to include in the image analysis by left clicking, finish with enter.


--------------------------------------
Scale found with 33 keypoint matches
--------------------------------------


     diameter  area
idx                
1          96  1429
2         109  2141
3         117  4116
4         101  2650
5         102  2217
..        ...   ...
16         98  1778
17         96  2095
18        119  2648
19        111  1722
20         95  1706

[20 rows x 2 columns]


----------------------------------------------------------------
Found 26 objects in isopods_multiple_1.jpg:
  ==> 20 are valid objects
    - 4 are not bigger than minimum diameter
    - 6 are not bigger than minimum area
----------------------------------------------------------------

Mark the outline of your arena, i.e. what you want to include in the image analysis by left clicking, finish with enter.


--------------------------------------
Scale found with 28 keyp

## object finder - variable background <a name="object4"></a>

In this example, the background is not homogenous enough for the standard thresholding algorithm, so we need to use different settings.

In [29]:
my_proj = pp.project_maker("images", include=["bug2"])



----------------------------------------------------------------
Project settings - "My project, 2019-02-16":


Image directory: images
Mode: dir
Filetypes: []
Include:['bug2']
Exclude: []
----------------------------------------------------------------
Filenames: 
['bug2.jpg']


In [30]:
of = pp.object_finder(image=my_proj.filepaths[0])
results = of.find_objects(mode="single", blur1=10) # does not work

Warning - no scale specified
----------------------------------------------------------------
Found following object in bug2.jpg:
----------------------------------------------------------------
     diameter     area
idx                   
1        1684  1368416


The algorithm fails because the image is full of (to us) invisible gradients that make and inhomogenous background. We can make them visible if we change the algorithm, and binarize:

In [31]:
results = of.find_objects(mode="single", method=["adaptive", 99,1], show=False) 
pp.show_img(of.thresh)

Warning - no scale specified
----------------------------------------------------------------
Found following object in bug2.jpg:
----------------------------------------------------------------
     diameter   area
idx                 
1         382  10097


In [32]:
# and now with proper output
results = of.find_objects(mode="single", blur1=10, method=["adaptive", 99,3]) # does work better

Warning - no scale specified
----------------------------------------------------------------
Found following object in bug2.jpg:
----------------------------------------------------------------
     diameter  area
idx                
1         174  5976


In [33]:
pp.save_csv(results, my_proj.filenames[0], "images_out")
pp.save_img(of.image_processed, my_proj.filenames[0], "images_out", resize=0.5)

END - more tutorials to come! If you have questions in the meantime, email me. 